In [1]:
import pickle
import pandas as pd
import numpy as np
import os
d = "raw/pk/"
files = sorted([d + f for f in os.listdir(d) if f.endswith("lpow")])
print(files)

['raw/pk/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9001_flip_v3.0_n200rand.lpow', 'raw/pk/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9001_v3.0_n200rand.lpow', 'raw/pk/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9002_flip_v3.0_n200rand.lpow', 'raw/pk/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9002_v3.0_n200rand.lpow', 'raw/pk/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9003_flip_v3.0_n200rand.lpow', 'raw/pk/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9003_v3.0_n200rand.lpow', 'raw/pk/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9004_flip_v3.0_n200rand.lpow', 'raw/pk/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9004_v3.0_n200rand.lpow', 'raw/pk/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9005_flip_v3.0_n200rand.lpow', 'raw/pk/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9005_v3.0_n200rand.lpow', 'raw/pk/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9006_flip_v3.0_n200rand.lpow', 'raw/pk/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9006_v3.0_n200rand.lpow', 'raw/pk/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9007_flip_v3.0_n200rand.lpow', 'raw/pk/mock_gsafe0DR7VACNGCm_abM21.2_gr8

In [12]:
def getdf(loc):
    df = pd.read_csv(loc, comment="#", delim_whitespace=True, names=["i", "k", "pk", "nk"])
    mask = (df["nk"] > 0) & (df["k"] <= 0.4)
    masked = df.loc[mask, ["k", "pk", "nk"]]
    return masked.astype(np.float32)

def get_winfits(directory):
    files = sorted([directory + f for f in os.listdir(directory) if "winfit" in f])
    print(files)
    winfits = {}
    for winfit_file in files:
        step_size = int(winfit_file.split(".")[-2].split("_")[-1])
        print(f"Loading winfit from {winfit_file}")
        matrix = np.genfromtxt(winfit_file, skip_header=4)
        res = {}
        winfits[step_size] = res
        res["w_ks_input"] = matrix[:, 0]
        res["w_k0_scale"] = matrix[:, 1]
        res["w_transform"] = matrix[:, 2:]/(np.sum(matrix[:, 2:], axis=0))
        # God I am sorry for doing this manually but the file format is... tricky
        with open(winfit_file, "r") as f:
            res["w_ks_output"] = np.array([float(x) for x in f.readlines()[2].split()[1:]])
    return winfits

def get_winpk(directory):
    files = sorted([directory + f for f in os.listdir(directory) if "lwin" in f])
    assert len(files) == 1, str(files)
    print(files[0])
    data = np.genfromtxt(files[0])
    return data

res = {f.lower(): getdf(f) for f in files}
split = {
    "pre-recon": [v for k, v in res.items() if "recon" not in k],
    "post-recon": [v for k, v in res.items() if "recon" in k],
    "cosmology": {"om": 0.31, "h0": 0.67, "z": 0.15, "ob": 0.0491, "ns": 0.96, "reconscale": 21.21},
    "name": "SDSS MGS DR7 Pk",
    "winfit": get_winfits("./"),
    "winpk": get_winpk("./")
}

['./bin0_winfit_10.txt', './bin0_winfit_5.txt', './bin0_winfit_7.txt']
Loading winfit from ./bin0_winfit_10.txt
Loading winfit from ./bin0_winfit_5.txt
Loading winfit from ./bin0_winfit_7.txt
./lwin.txt


In [13]:
with open("../sdss_dr7_pk.pkl", "wb") as f:
    pickle.dump(split, f)

In [10]:
split["winfit"]

{5: {'w_k0_scale': array([1.840500e-03, 1.086400e-02, 2.961620e-02, 9.386120e-02,
         1.747046e-01, 8.850280e-02, 5.837370e-02, 5.935300e-02,
         5.024720e-02, 5.671860e-02, 5.702880e-02, 4.443050e-02,
         3.341730e-02, 2.467330e-02, 1.643030e-02, 1.094480e-02,
         8.146400e-03, 7.140100e-03, 6.849000e-03, 6.558600e-03,
         6.396200e-03, 6.518300e-03, 6.695300e-03, 6.653900e-03,
         6.457900e-03, 6.201600e-03, 5.883200e-03, 5.433100e-03,
         4.876400e-03, 4.312800e-03, 3.816900e-03, 3.370200e-03,
         2.934200e-03, 2.505300e-03, 2.129200e-03, 1.861100e-03,
         1.721600e-03, 1.692800e-03, 1.742300e-03, 1.834800e-03,
         1.947300e-03, 2.050100e-03, 2.115600e-03, 2.115400e-03,
         2.035800e-03, 1.881100e-03, 1.682700e-03, 1.461200e-03,
         1.240700e-03, 1.043700e-03, 8.883000e-04, 7.792000e-04,
         7.200000e-04, 7.089000e-04, 7.372000e-04, 7.931000e-04,
         8.669000e-04, 9.425000e-04, 1.003000e-03, 1.037700e-03,
        

In [4]:
a.shape

(249, 3)